In [ ]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import word2vec

In [ ]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
sample_submission = pd.read_csv("../data/sample_submission.csv")

In [ ]:
train_df

In [ ]:
train_df.location.value_counts(dropna=False,normalize=True)

In [ ]:
train_df.keyword.value_counts(dropna=False,normalize=True)

In [ ]:
train_df.target.value_counts(dropna=False,normalize=True)

In [ ]:
import nltk  

## remove stop words

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer 
import re
lem = WordNetLemmatizer()


In [ ]:
def preprocess_sentence(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    regex = re.compile('([^\s\w]|_)+')
    cleantext = regex.sub('', cleantext).lower()
    tokens = word_tokenize(cleantext)
    tokens_normalized = [word.lower() for word in tokens if len(word)>2 and word.lower() not in stopwords.words('english')]
    tokens_lemmatized = [lem.lemmatize(word) for word in tokens_normalized]
    return tokens_lemmatized

In [ ]:
train_df["tokens"] = train_df["text"].map(lambda x: preprocess_sentence(x))

In [ ]:
cv

In [ ]:
terms_frequency = pd.DataFrame({"word": [item for sublist in np.array(train_df.tokens).tolist() for item in sublist ]})

In [ ]:
corpus = terms_frequency.drop_duplicates().word.tolist()

In [ ]:
model = word2vec.Word2Vec(train_df.tokens, size=100, window=20, min_count=2, workers=4)

In [ ]:
from sklearn import feature_extraction

In [ ]:
vectorizer = feature_extraction.text.CountVectorizer(max_features=10000, ngram_range=(1,2))